In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

In [2]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：10



,No,Type,Part Number,Title,Application Summary,Picture,Url,Subtitle,Json_Src,Json_Info
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
1,2,Windshield Wiper Motor,40-10000,Windshield Wiper Motor - 40-10000,Hummer H3 2010-06,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
2,3,Windshield Wiper Motor,40-10005,Windshield Wiper Motor - 40-10005,,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
3,4,Windshield Wiper Motor,40-10006,Windshield Wiper Motor - 40-10006,Buick Enclave 2017-08,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
4,5,Windshield Wiper Motor,40-10007,Windshield Wiper Motor - 40-10007,Chevrolet Malibu 2007-05,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
5,6,Windshield Wiper Motor,40-10009,Windshield Wiper Motor - 40-10009,"Chevrolet Suburban 1500 2000, Suburban 2500 20...",,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."
6,7,Windshield Wiper Motor,40-1001,Windshield Wiper Motor - 40-1001,Chevrolet Prizm 2002-98; Geo Prizm 1997-93; To...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1001..."
7,8,Windshield Wiper Motor,40-10013,Windshield Wiper Motor - 40-10013,Chevrolet Captiva Sport 2015-08; Saturn Vue 20...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1001..."
8,9,Windshield Wiper Motor,40-10014,Windshield Wiper Motor - 40-10014,Chevrolet Captiva Sport 2010-08; Saturn Vue 20...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1001..."
9,10,Windshield Wiper Motor,40-10015,Windshield Wiper Motor - 40-10015,Geo Metro 1994-89; Pontiac Firefly 1994 1991-89,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1001..."


In [4]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                                     1
 Type                                              Windshield Wiper Motor
 Part Number                                                      40-1000
 Title                                   Windshield Wiper Motor - 40-1000
 Application Summary    Chevrolet Camaro 1998-93; Pontiac Firebird 199...
 Picture                                                                 
 Url                    https://www.cardone.com/product/windshield-wip...
 Subtitle                                                   CARDONE Reman
 Json_Src               {"0": "https://cdn11.bigcommerce.com/s-z9vhe3o...
 Json_Info              {"0": {"productDeskExternalKey": "CPCX|40-1000...
 Name: 0, dtype: object)

In [5]:
import time

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request, url_check = f'''https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:{crawler_series['Part Number']}&size=1&useCache=false&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

len(resp.json())

1
2


1

In [6]:
list_dict = resp.json()[0]['CrossReferences'] if 'CrossReferences' in resp.json()[0] else []
list_reference = [dict_['reference'].strip() for dict_ in list_dict if dict_['referenceType'].strip() == 'OE Part']
oem = ';'.join(list(dict.fromkeys(list_reference)))

oem

'22154913;22110039'

In [7]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Type': crawler_series['Type'],
                          'Part Number': crawler_series['Part Number'],
                          'Title': crawler_series['Title'],
                          'Application Summary': crawler_series['Application Summary'],
                          'OEM': oem,
                          'Picture': crawler_series['Picture'],
                          'Url': crawler_series['Url'],
                          'Subtitle': crawler_series['Subtitle'],
                          'Json_Src': crawler_series['Json_Src'],
                          'Json_Info': crawler_series['Json_Info']})

list_dict_correct

[{'No': 1,
  'Type': 'Windshield Wiper Motor',
  'Part Number': '40-1000',
  'Title': 'Windshield Wiper Motor - 40-1000',
  'Application Summary': 'Chevrolet Camaro 1998-93; Pontiac Firebird 1998-93',
  'OEM': '22154913;22110039',
  'Picture': '',
  'Url': 'https://www.cardone.com/product/windshield-wiper-motor-40-1000',
  'Subtitle': 'CARDONE Reman',
  'Json_Src': '{"0": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488626/482__00992.1687973871.jpg?c=1", "1": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488628/482__97909.1687973872.jpg?c=1", "2": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488630/482__09398.1687973874.jpg?c=1", "3": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488632/482__69563.1687973875.jpg?c=1", "4": "https://cdn11.bigcommerce.com/s-z9vhe3o238/images/stencil/1280x1280/products/82816/5488635/482__93868.168797

In [8]:
crawler_status = True

crawler_status

True

In [9]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                                     1
 Type                                              Windshield Wiper Motor
 Part Number                                                      40-1000
 Title                                   Windshield Wiper Motor - 40-1000
 Application Summary    Chevrolet Camaro 1998-93; Pontiac Firebird 199...
 Picture                                                                 
 Url                    https://www.cardone.com/product/windshield-wip...
 Subtitle                                                   CARDONE Reman
 Json_Src               {"0": "https://cdn11.bigcommerce.com/s-z9vhe3o...
 Json_Info              {"0": {"productDeskExternalKey": "CPCX|40-1000...
 Request_Url            https://productdesk-api.cellacore.net/legacy/c...
 Check_Url              https://www.cardone.com/product/windshield-wip...
 Name: 0, dtype: object]

In [10]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_oem.xlsx', index=False)

df_correct

,No,Type,Part Number,Title,Application Summary,OEM,Picture,Url,Subtitle,Json_Src,Json_Info
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,22154913;22110039,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000..."


In [11]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_oem_error.xlsx', index=False)

df_error

,No,Type,Part Number,Title,Application Summary,Picture,Url,Subtitle,Json_Src,Json_Info,Request_Url,Check_Url
0,1,Windshield Wiper Motor,40-1000,Windshield Wiper Motor - 40-1000,Chevrolet Camaro 1998-93; Pontiac Firebird 199...,,https://www.cardone.com/product/windshield-wip...,CARDONE Reman,"{""0"": ""https://cdn11.bigcommerce.com/s-z9vhe3o...","{""0"": {""productDeskExternalKey"": ""CPCX|40-1000...",https://productdesk-api.cellacore.net/legacy/c...,https://www.cardone.com/product/windshield-wip...
